# Análisis de sentimientos(Azure Text Analytics)

In [7]:
import pandas as pd 
import time

In [7]:
# Instaalmos libreria 
#! pip install azure-ai-textanalytics --pre

In [8]:
# Definimos las variables de conexion.Estas las tenemso que sacar de nuestro portal de Azure.

key = "Rellenar con claves personales"
endpoint = "Rellenar con claves personales"

In [11]:
#Cree una función para crear una instancia del objeto TextAnalyticsClient con los objetos key y endpoint creados anteriormente. Luego, cree un cliente.

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [46]:
#Importamos csv que quremso Analizar.En este caso utilizaremso el descargado anteriormente.
df = pd.read_csv('LaIslaDeLastenataciones5-tweets-limpio-completo.csv')

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66744 entries, 0 to 66743
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    66744 non-null  int64 
 1   Unnamed: 0.1  66744 non-null  int64 
 2   Datetime      66744 non-null  object
 3   Tweet Id      66744 non-null  int64 
 4   Creacion      66744 non-null  object
 5   Username      66744 non-null  object
 6   Description   57715 non-null  object
 7   Location      40526 non-null  object
 8   Following     66744 non-null  int64 
 9   Followers     66744 non-null  int64 
 10  Total Tweets  66744 non-null  int64 
 11  RT            66744 non-null  int64 
 12  Text          66744 non-null  object
 13  Hashtags      66744 non-null  object
dtypes: int64(7), object(7)
memory usage: 7.1+ MB


In [48]:
df.tail()

,Unnamed: 0,Unnamed: 0.1,Datetime,Tweet Id,Creacion,Username,Description,Location,Following,Followers,Total Tweets,RT,Text,Hashtags
66739,66739,66864,2021-02-11 21:20:02,1359975507085910016,2020-10-12 21:57:55,90srosinkranz,"thank u, next — track 5",she¡!her,421,419,2855,0,ay lucia mi amor lo que vas a llorar yo me enc...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66740,66740,66865,2021-02-11 21:20:02,1359975506393918976,2019-05-06 13:25:17,EscandaloNando,"Eurofan, adicto a los realitys y fan de Pokemo...","Badajoz, España",1747,2102,10826,0,"de las 3 ediciones, es la que menos me está gu...","[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66741,66741,66866,2021-02-11 21:20:01,1359975502639955968,2016-02-07 13:23:26,mariiamc02,"Si me quieres, dame palomitas",España,321,392,38655,1,todos somos lara. todos queremos tocar el pelo...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66742,66742,66867,2021-02-11 21:20:00,1359975497237691904,2018-08-22 20:58:39,whateverrcloud,NaN,NaN,347,304,2940,1,uf qué guapa lara tentación la isla,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66743,66743,66868,2021-02-11 21:20:00,1359975496231058944,2012-12-22 23:30:21,Alba1169,Jugadora del VEC Barcelona,NaN,136,251,2371,0,he sido capaz de no tragarme ningún spoiler po...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."


In [52]:
#Pasamos la columna de Text para analizar.

df['Text']

0        es que no me creo que sea la tercera temporada...
1        marina: ¿y en que clase de juegos os coméis la...
2        vacunilla president!!! otegui lendakari !! sán...
3            hugo me deja sin palabras, en serio.  la isla
4        lucia va a rerollear a dps desde tank. desde t...
                               ...                        
66739    ay lucia mi amor lo que vas a llorar yo me enc...
66740    de las 3 ediciones, es la que menos me está gu...
66741    todos somos lara. todos queremos tocar el pelo...
66742                 uf qué guapa lara tentación  la isla
66743    he sido capaz de no tragarme ningún spoiler po...
Name: Text, Length: 66744, dtype: object

In [53]:
#Creamos función para llamar a la API.

def sentiment_analysis_example(client, tweet):

    documents = [tweet]
    response = client.analyze_sentiment(documents=documents)[0]
    return response


In [56]:
#Creamos funcion para iterar todos los tweet y crear DF.

tweets_df=[] 
sentence_df=[] 

#Con el primer for sacaremos el sentimiento del tweet al completo
for i in range(df['Text'].count()):
    try:
        tweet = df['Text'][i]
        response = sentiment_analysis_example(client, tweet)
    ''' print("Document Sentiment: {}".format(response.sentiment))
        print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            response.confidence_scores.positive,
            response.confidence_scores.neutral,
            response.confidence_scores.negative,
        ))'''
        
        # Atributos del Tweet con los que rellenaremos el primr DF.
        tweets_df.append([tweet ,response.sentiment,response.confidence_scores.positive, response.confidence_scores.neutral,response.confidence_scores.negative])
        #print(tweets_df)

        #Con este segundo for sacamos el sentimiento por frases.
        for idx, sentence in enumerate(response.sentences):
            '''print("Sentence: {}".format(sentence.text))
            print("Sentence {} sentiment: {}".format(idx+1, sentence.sentiment))
            print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))'''
            
            #Atributos de la frase  con las que rellenaremos segundo DF.
            sentence_df.append([sentence.text,sentence.sentiment,sentence.confidence_scores.positive, sentence.confidence_scores.neutral,sentence.confidence_scores.negative])
            #print(sentence_df)


        print('Vamos a comprobar si tiene que esperar',i)
        
        # Cremos DF
        tweets_df_final =pd.DataFrame(tweets_df,columns=['Text','Total sentimiento','Positivo','Neutral','Negativo'])         
        sentence_df_final =pd.DataFrame(sentence_df,columns=['Text','Total sentimiento','Positivo','Neutral','Negativo'])

        #Guardamos DF
        tweets_df_final.to_csv('Sentimiento-tweets-5.csv', sep=',', index = False)
        sentence_df_final.to_csv('Sentimientos-frases-5.csv', sep=',', index = False)
            #comprobar limite
        time.sleep(0.25)

            #time.sleep(0.25) #Limite 
    except Exception:
        print("Error en la llamada:" + str(i))
        pass

#Podemos sacarlo fuera, pero si falla durante el proceso pierdo toda la información
#tweets_df_final =pd.DataFrame(tweets_df,columns=['Text','Total sentimiento','Positivo','Neutral','Negativo'])         
#sentence_df_final =pd.DataFrame(sentence_df,columns=['Text','Total sentimiento','Positivo','Neutral','Negativo'])


#tweets_df_final.to_csv('Sentimiento-tweets-5.csv', sep=',', index = False)
#sentence_df_final.to_csv('Sentimientos-frases-5.csv', sep=',', index = False)

Vamos a comprobar si tiene que esperar 0
Vamos a comprobar si tiene que esperar 1
Vamos a comprobar si tiene que esperar 2
Vamos a comprobar si tiene que esperar 3
Vamos a comprobar si tiene que esperar 4
Vamos a comprobar si tiene que esperar 5
Vamos a comprobar si tiene que esperar 6
Vamos a comprobar si tiene que esperar 7
Vamos a comprobar si tiene que esperar 8
Vamos a comprobar si tiene que esperar 9
Vamos a comprobar si tiene que esperar 10
Vamos a comprobar si tiene que esperar 11
Vamos a comprobar si tiene que esperar 12
Vamos a comprobar si tiene que esperar 13
Vamos a comprobar si tiene que esperar 14
Vamos a comprobar si tiene que esperar 15
Vamos a comprobar si tiene que esperar 16
Vamos a comprobar si tiene que esperar 17
Vamos a comprobar si tiene que esperar 18
Vamos a comprobar si tiene que esperar 19
Vamos a comprobar si tiene que esperar 20
Vamos a comprobar si tiene que esperar 21
Vamos a comprobar si tiene que esperar 22
Vamos a comprobar si tiene que esperar 23
Va

In [57]:
#Importamso csv que hemos obtenido 
import pandas as pd 
df1 = pd.read_csv('Sentimiento-tweets-5.csv')
df2 =pd.read_csv('Sentimientos-frases-5.csv')

In [59]:
df1['Total sentimiento'].value_counts()

positive    25079
negative    24888
neutral      8939
mixed        7835
Name: Total sentimiento, dtype: int64

In [61]:
df1.head()

,Text,Total sentimiento,Positivo,Neutral,Negativo
0,es que no me creo que sea la tercera temporada...,mixed,0.44,0.05,0.51
1,marina: ¿y en que clase de juegos os coméis la...,positive,0.91,0.02,0.07
2,vacunilla president!!! otegui lendakari !! sán...,positive,0.77,0.13,0.10
3,"hugo me deja sin palabras, en serio. la isla",mixed,0.23,0.20,0.57
4,lucia va a rerollear a dps desde tank. desde t...,positive,0.45,0.43,0.12


In [62]:

df1.rename({ 'Text': 'Text_1'}, axis=1, inplace=True)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66744 entries, 0 to 66743
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    66744 non-null  int64 
 1   Unnamed: 0.1  66744 non-null  int64 
 2   Datetime      66744 non-null  object
 3   Tweet Id      66744 non-null  int64 
 4   Creacion      66744 non-null  object
 5   Username      66744 non-null  object
 6   Description   57715 non-null  object
 7   Location      40526 non-null  object
 8   Following     66744 non-null  int64 
 9   Followers     66744 non-null  int64 
 10  Total Tweets  66744 non-null  int64 
 11  RT            66744 non-null  int64 
 12  Text          66744 non-null  object
 13  Hashtags      66744 non-null  object
dtypes: int64(7), object(7)
memory usage: 7.1+ MB


In [64]:
#Concatenamos df para tener toda la informacióm
df_concat =pd.concat([df,df1],axis =1) 

In [65]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66744 entries, 0 to 66743
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         66744 non-null  int64  
 1   Unnamed: 0.1       66744 non-null  int64  
 2   Datetime           66744 non-null  object 
 3   Tweet Id           66744 non-null  int64  
 4   Creacion           66744 non-null  object 
 5   Username           66744 non-null  object 
 6   Description        57715 non-null  object 
 7   Location           40526 non-null  object 
 8   Following          66744 non-null  int64  
 9   Followers          66744 non-null  int64  
 10  Total Tweets       66744 non-null  int64  
 11  RT                 66744 non-null  int64  
 12  Text               66744 non-null  object 
 13  Hashtags           66744 non-null  object 
 14  Text_1             66741 non-null  object 
 15  Total sentimiento  66741 non-null  object 
 16  Positivo           667

In [67]:
df_concat.drop(['Unnamed: 0','Text_1',],1, inplace = True)

In [68]:
#Guardamos documento en la versión que queramos.
df_concat.to_excel('DF_completo_final_IT5.xlsx')